# Segmenting and Clustering Neighborhoods in Toronto Part 3

Obtaining the postal codes and transform into panda dataframe

In [1]:
!pip3 install pandas lxml

In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

toronto_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


The dataframe should consist of three columns: PostalCode, Borough, and Neighborhood. 
Therefore we'll rename "Postal Code" to "PostalCode" and "Neighbourhood" to "Neighborhood"

In [3]:
toronto_df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
toronto_df.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
list(toronto_df.columns.values)

['PostalCode', 'Borough', 'Neighborhood']

Only process the cells that have an assigned borough

In [5]:
toronto_df.shape

(180, 3)

In [6]:
toronto_df_filtered = toronto_df[toronto_df['Borough'] != 'Not assigned']
toronto_df_filtered.reset_index(drop=True, inplace=True)

toronto_df_filtered.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
toronto_df_filtered.shape

(103, 3)

So, we have reduced our row size. Now, we are checking, if there are any  duplicated postal codes:

In [8]:
toronto_df_filtered['PostalCode'].value_counts()

M6H    1
M9B    1
M6G    1
M2L    1
M5C    1
M2N    1
M4T    1
M5L    1
M1S    1
M4X    1
M6E    1
M1R    1
M1V    1
M6L    1
M5H    1
M9C    1
M3B    1
M3A    1
M6C    1
M9P    1
M5W    1
M9V    1
M9N    1
M1T    1
M4P    1
M5P    1
M1P    1
M9A    1
M4B    1
M4G    1
M4J    1
M6N    1
M1K    1
M5B    1
M1B    1
M2P    1
M1H    1
M3C    1
M9L    1
M4A    1
M6P    1
M4H    1
M7A    1
M2R    1
M1G    1
M5N    1
M4C    1
M5X    1
M4S    1
M8Y    1
M4M    1
M3J    1
M1L    1
M3K    1
M2M    1
M1J    1
M5M    1
M6S    1
M6B    1
M1W    1
M6A    1
M7Y    1
M6M    1
M9W    1
M1C    1
M9M    1
M5V    1
M6R    1
M2K    1
M8Z    1
M5K    1
M5R    1
M1M    1
M4V    1
M4K    1
M8X    1
M5J    1
M1E    1
M3L    1
M4W    1
M6K    1
M4N    1
M3H    1
M4R    1
M6J    1
M4E    1
M5A    1
M2H    1
M5E    1
M5G    1
M4L    1
M1X    1
M3M    1
M4Y    1
M1N    1
M7R    1
M8W    1
M5T    1
M2J    1
M5S    1
M8V    1
M3N    1
M9R    1
Name: PostalCode, dtype: int64

Since the row size is the same, all postal codes are unique.

Assign all "Not assigned" neighborhood to the borough ones. Let's check these rows:

In [9]:
toronto_df_filtered[toronto_df_filtered['Neighborhood'] == 'Not assigned'].value_counts()

Series([], dtype: int64)

Which means there is any not assigned Neighbourhood value. Already, formated accordingly in the wiki page

In [10]:
row, col = toronto_df_filtered.shape

print('The row size is', row, 'and the column size is', col)

The row size is 103 and the column size is 3


Now, we need to get the latitude and the longitude coordinates of each neighborhood. First we'll try to read from geogratis.gc.ca's API service. If we get a failure, both lat and lon values will be obtained from the provided csv URL: http://cocl.us/Geospatial_data

In [11]:
geo_df = pd.read_csv('https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv')

print('shape', geo_df.shape)

geo_df.head()

shape (103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
import requests

def getLatLang(post_code):
    lat = 0
    lon = 0
    try:
        geo_json = requests.get('http://geogratis.gc.ca/services/geolocation/en/locate?q=' + post_code).json()
        lat = geo_json[0]['geometry']['coordinates'][1]
        lon = geo_json[0]['geometry']['coordinates'][0]
    except:
        # Got an error. reading from csv.
        print('Error for', post_code, 'retrying with backup service')
        geo_df = pd.read_csv('https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv')
        lat = geo_df[geo_df['Postal Code']==post_code]['Latitude'].values[0]
        lon = geo_df[geo_df['Postal Code']==post_code]['Longitude'].values[0]
        
    return lat, lon

lat, lon = getLatLang('M1B')

print('M1B postal code geo','Latitude:',lat,'Longitude:',lon)

M1B postal code geo Latitude: 43.809444 Longitude: -79.193321


Now, we will append the latitute and longitude coordinates to the filterd dataframe.

In [13]:
lat_list =[]
lon_list = []

for pc in toronto_df_filtered['PostalCode']:
    lat,lon = getLatLang(pc)
    lat_list.append(lat)
    lon_list.append(lon)

toronto_df_filtered = toronto_df_filtered.assign(Latitude=pd.Series(lat_list))
toronto_df_filtered = toronto_df_filtered.assign(Longitude=pd.Series(lon_list))

toronto_df_filtered.head()

Error for M7R retrying with backup service


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752804,-79.329590
1,M4A,North York,Victoria Village,43.723358,-79.312927
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.647398,-79.352920
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.724842,-79.451431
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.663681,-79.392097


In [14]:
!pip3 install geopy

In [15]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import folium

import numpy as np # library to handle data in a vectorized manner

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

## Create map of Toronto

Now, we'll get Toronto's central location

In [16]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [17]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_filtered['Latitude'], toronto_df_filtered['Longitude'], toronto_df_filtered['Borough'], toronto_df_filtered['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Filter only boroughs that contains "Toronto" 

In [18]:
toronto_data = toronto_df_filtered[toronto_df_filtered['Borough'].str.contains('Toronto')].reset_index(drop=True)

toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.647398,-79.352920
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.663681,-79.392097
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.658165,-79.378456
3,M5C,Downtown Toronto,St. James Town,43.651802,-79.375801
4,M4E,East Toronto,The Beaches,43.678986,-79.296967


In [19]:
toronto_data.shape

(39, 5)

## Explore the first neighborhood

In [20]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.647398, -79.3529205.


## Set Foursquare credentials

In [21]:
import os

CLIENT_ID = ''
CLIENT_SECRET = ''
ACCESS_TOKEN = ''
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

## Now, let's get the top 100 venues that are in Regent Park, Harbourfront within a radius of 500 meters.

In [22]:
LIMIT = 100
radius = 500

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-22-80a5aa4d802c>:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,BATLgrounds,Athletics & Sports,43.647088,-79.351306
1,Cirque du Soleil - LUZIA,Circus,43.644791,-79.351000
2,Lower Don Recreational Trail,Athletics & Sports,43.648414,-79.354716
3,Cherry Street Bar-B-Que,BBQ Joint,43.645644,-79.352380
4,The Keating Channel Pub & Grill,Pub,43.647709,-79.353369


Returned venues number by Foursquare:

In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


Explore Neighborhoods in Toronto

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues (names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [25]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.647398,-79.35292,BATLgrounds,43.647088,-79.351306,Athletics & Sports
1,"Regent Park, Harbourfront",43.647398,-79.35292,Cirque du Soleil - LUZIA,43.644791,-79.351000,Circus
2,"Regent Park, Harbourfront",43.647398,-79.35292,Lower Don Recreational Trail,43.648414,-79.354716,Athletics & Sports
3,"Regent Park, Harbourfront",43.647398,-79.35292,Cherry Street Bar-B-Que,43.645644,-79.352380,BBQ Joint
4,"Regent Park, Harbourfront",43.647398,-79.35292,The Keating Channel Pub & Grill,43.647709,-79.353369,Pub


In [26]:
print(toronto_venues.shape)

(1529, 7)


In [27]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,77,77,77,77,77,77
"Brockton, Parkdale Village, Exhibition Place",18,18,18,18,18,18
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",7,7,7,7,7,7
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",56,56,56,56,56,56
Central Bay Street,76,76,76,76,76,76
Christie,14,14,14,14,14,14
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,30,30,30,30,30,30


Number of unique venues:

In [28]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 226 uniques categories.


## Analyze Each Neighborhood

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,College Quad,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,College Quad,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.012987,0.0,0.0,0.012987,0.012987,0.012987,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.012987,0.025974,0.000000,0.000000,0.0,0.0,0.025974,0.0,0.0,0.012987,0.0,0.000000,0.0,0.0,0.025974,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.025974,0.0,0.0,0.000000,0.025974,0.0,0.012987,0.0,0.000000,0.000000,0.025974,0.077922,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.012987,0.000000,0.012987,0.0,0.0,0.012987,0.0,0.025974,0.0,0.0,0.0,0.012987,0.012987,0.000000,0.0,0.012987,0.000000,0.0,0.0,0.0,0.000000,0.0,0.012987,0.000000,0.0,0.0,0.0,0.0,0.025974,0.012987,0.0,0.012987,0.0,0.0,0.000000,0.012987,0.012987,0.012987,0.0,0.000000,0.0,0.0,0.0,0.012987,0.0,0.0,0.0,0.0,0.0,0.012987,0.0,0.012987,0.000000,0.000000,0.0,0.000000,

Top 5 most common venues

In [31]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1               Hotel  0.04
2  Seafood Restaurant  0.04
3          Restaurant  0.04
4              Bakery  0.03


----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0         Soccer Stadium  0.17
1  Performing Arts Venue  0.11
2           Intersection  0.06
3             Food Truck  0.06
4           Climbing Gym  0.06


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                 venue  freq
0       Hardware Store  0.29
1     Asian Restaurant  0.14
2       Discount Store  0.14
3  Sporting Goods Shop  0.14
4         Dessert Shop  0.14


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.07
2                Café  0.07
3          Restaurant  0.05
4      Sandwich Place  0.04


----Central Bay

Create the new dataframe and display the top 10 venues for each neighborhood.

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Hotel,Restaurant,Park,Farmers Market,Breakfast Spot,Italian Restaurant,Café,Japanese Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Soccer Stadium,Performing Arts Venue,Theme Park,Restaurant,Food Court,Food Truck,Intersection,Climbing Gym,Athletics & Sports,Theater
2,"Business reply mail Processing Centre, South C...",Hardware Store,Discount Store,Sporting Goods Shop,Asian Restaurant,Pub,Dessert Shop,Falafel Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Café,Italian Restaurant,Restaurant,Park,Caribbean Restaurant,Sandwich Place,Mexican Restaurant,Market,Gym
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Middle Eastern Restaurant,Department Store,Hotel,Bubble Tea Shop,Restaurant,Café,Sushi Restaurant


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 4 clusters.

In [33]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.647398,-79.352920,2,Athletics & Sports,Circus,Harbor / Marina,Pub,Trail,BBQ Joint,Boat or Ferry,Wings Joint,Dog Run,Doner Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.663681,-79.392097,2,Coffee Shop,Park,Chinese Restaurant,College Theater,Art Museum,Sushi Restaurant,Theater,Doner Restaurant,Distribution Center,Dog Run
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.658165,-79.378456,2,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Falafel Restaurant,Lounge,Italian Restaurant,Diner,Pizza Place,Fast Food Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651802,-79.375801,2,Coffee Shop,Café,Cocktail Bar,Seafood Restaurant,Gastropub,American Restaurant,Cosmetics Shop,Restaurant,Moroccan Restaurant,Farmers Market
4,M4E,East Toronto,The Beaches,43.678986,-79.296967,2,Church,Health Food Store,Park,Pizza Place,Trail,Coffee Shop,Pub,Asian Restaurant,Doner Restaurant,Discount Store


Visualize the resulting clusters.

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Now, we can determine the discriminating venue categories that distinguish each cluster.

In [36]:
for i in range(0,kclusters):
    print("Cluster: ", i)
    print(toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]])
    print("********************************************************************************")

Cluster:  0
             Borough  Cluster Labels 1st Most Common Venue  \
10  Downtown Toronto               0         Boat or Ferry   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
10           Wings Joint  Fast Food Restaurant                  Farm   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
10    Falafel Restaurant  Ethiopian Restaurant           Escape Room   

   8th Most Common Venue        9th Most Common Venue 10th Most Common Venue  
10     Electronics Store  Eastern European Restaurant    Dumpling Restaurant  
********************************************************************************
Cluster:  1
             Borough  Cluster Labels 1st Most Common Venue  \
18   Central Toronto               1    Photography Studio   
21   Central Toronto               1                  Park   
23   Central Toronto               1                  Park   
28      West Toronto               1                  Park   
33  Downtown